# Growth enhancing additives on CGXII medium

This notebook holds growth simulation with different growth enhancing metabolites for all strains.

*Author: Famke Baeuerle*

In [1]:
# imports
from libsbml import *
import refinegems as rg
import pandas as pd
import numpy as np

# strain naming
STRAINS_LAB = {'14':'TS', 
            '15':'1197',
            '16':'1115',
            '17':'1116',
            }

CPLEX is available but could not load with error:
  Traceback (most recent call last):
    File "/Users/baeuerle/Library/Python/3.9/lib/python/site-packages/optlang/__init__.py", line 43, in <module>
      from optlang import cplex_interface
    File "/Users/baeuerle/Library/Python/3.9/lib/python/site-packages/optlang/cplex_interface.py", line 32, in <module>
      from cplex.exceptions import CplexSolverError
  ModuleNotFoundError: No module named 'cplex.exceptions'


First we will define functions which allow us to conduct reproducible simulation without needing to copy paste the routines.

In [2]:
def find_additives(model: Model, base_medium: dict) -> pd.DataFrame:
    """_summary_

    Args:
        model (Model): Model loaded with libSBML
        base_medium (dict): holding all exchanges that are defined for the medium and present in the model

    Returns:
        pd.DataFrame: exchanges of metabolites that enhace growth and their enhancement
    """
    with model:
        medium = model.medium
        model.medium = base_medium
        sol = model.optimize()      
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate without additives: ' + str(round(base_growth, 4)))
    print('Doubling time without additives: ' + str(round(dt, 2)) + ' min')
        
    enhancement = {}
    for ex in list(model.exchanges):
        if ex.id not in base_medium.keys():
            with model:
                medium = model.medium
                base_medium.update({ex.id:10.0})
                model.medium = base_medium
                sol = model.optimize()
                if sol.objective_value > base_growth:
                    enhancement[ex.id] = sol.objective_value - base_growth
                base_medium.pop(ex.id)
    
    enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff']).sort_values(by=['diff'], ascending=False)
    
    return enh

def enhanced_growth(model: Model, base_medium: dict, new_metabs: list[str]):
    """_summary_

    Args:
        model (Model): Model loaded with libSBML
        base_medium (dict): holding all exchanges that are defined for the medium and present in the model
        new_metabs (list[str]): exchanges of metabolites that might enhance the growth
    """
    local_medium = base_medium
    with model:
        medium = model.medium
        for met in new_metabs:
            local_medium[met] = 10.0
        model.medium = local_medium
        sol = model.optimize()
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate with additives: ' + str(round(base_growth, 4)))
    print('Doubling time with additives: ' + str(round(dt, 2)) + ' min')

In this case we are interested in the CGXII medium and a possible growth enhancement by casamino acids.

In [3]:
CGXII = pd.read_csv('../../analysis/CGXII_base_medium/CGXII.csv').set_index('exchange').to_dict()['flux']
CasA = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='CasA']['BiGG'].to_list()

## Strain 1115

In [12]:
model = rg.io.load_model_cobra('../../models/iCstr1115FB23.xml')
CGXII_comp_16 = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_comp_16.csv').set_index('exchange').to_dict()['flux']

In [5]:
for rea in model.metabolites.get_by_id('malt_c').reactions:
    print(rea)

MTI: tre_c <=> malt_c
AMALT3: malt_c + maltpt_c --> glc__D_c + malthx_c
AMALT4: malt_c + malthx_c --> glc__D_c + malthp_c
AMALT1: malt_c + malttr_c --> glc__D_c + maltttr_c
MALTabc: atp_c + h2o_c + malt_e --> adp_c + h_c + malt_c + pi_c
AMALT2: malt_c + maltttr_c --> glc__D_c + maltpt_c
MLTG1: h2o_c + malttr_c --> glc__D_c + malt_c


In [11]:
print(model.id)
enh = find_additives(model, CGXII_comp_16)
enh.to_csv('../../analysis/additives/Cstr_16_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

iCstr1115FB23
Growth rate without additives: 0.8025
Doubling time without additives: 51.82 min


KeyboardInterrupt: 

Going through the given list I found that L-Arginine and N-Acetyl-D-glucosamine are available in the lab storage. Simulation with both metabolites leads to a growth value of 0.8 which corresponds to a doubling time of 52 Minutes.

In [13]:
enhanced = ['EX_arg__L_e', 'EX_acgam_e']
enhanced_growth(model, CGXII_comp_16, enhanced)

Growth rate with additives: 0.7042
Doubling time with additives: 59.06 min


In [9]:
model = rg.io.load_model_cobra('../../models/iCstr1115FB23.xml')
enhanced_growth(model, CGXII_comp_16, CasA)

Growth rate with additives: 0.8824
Doubling time with additives: 47.13 min


In [10]:
CGXII_base = CGXII_comp_16.copy()
for metab, flux in CGXII_comp_16.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base

{'EX_nh4_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_k_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_urea_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_h_e': 10.0}

In [1]:
model = rg.io.load_model_cobra('../../models/iCstr1115FB23.xml')
enhanced_growth(model, CGXII_base, CasA)

NameError: name 'rg' is not defined

## Strain 1197

In [12]:
model = rg.io.load_model_cobra('../../models/iCstr1197FB23.xml')
CGXII_comp_15 = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_comp_15.csv').set_index('exchange').to_dict()['flux']

In [13]:
print(model.id)
enh = find_additives(model, CGXII_comp_15)
enh.to_csv('../../analysis/additives/Cstr_15_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

iCstr1197FB23
Growth rate without additives: 0.5522
Doubling time without additives: 75.31 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 2.01175  |
| EX_malt_e   | 0.473353 |
| EX_tre_e    | 0.473353 |
| EX_sucr_e   | 0.473353 |
| EX_lcts_e   | 0.384599 |


Also shows enhancement by L-Arginine.

In [14]:
enhanced = ['EX_arg__L_e', 'EX_acgam_e']
enhanced_growth(model, CGXII_comp_15, enhanced)

Growth rate with additives: 0.5916
Doubling time with additives: 70.3 min


Lets test enhancement by Casamino acids.

In [15]:
model = rg.io.load_model_cobra('../../models/iCstr1197FB23.xml')
enhanced_growth(model, CGXII_comp_15, CasA)

Growth rate with additives: 0.6439
Doubling time with additives: 64.59 min


In [34]:
CGXII_base = CGXII_comp_15.copy()
for metab, flux in CGXII_comp_15.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)


In [35]:
model = rg.io.load_model_cobra('../../models/iCstr1197FB23.xml')
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.0
Doubling time with additives: inf min


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_13222/2110354515.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


## Strain TS

In [36]:
model = rg.io.load_model_cobra('../../models/iCstr1054FB23.xml')
CGXII_comp_14 = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_comp_14.csv').set_index('exchange').to_dict()['flux']

In [37]:
print(model.id)
enh = find_additives(model, CGXII_comp_14)
enh.to_csv('../../analysis/additives/Cstr_14_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

iCstr1054FB23
Growth rate without additives: 0.5485
Doubling time without additives: 75.82 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 1.99828  |
| EX_sucr_e   | 0.470184 |
| EX_tre_e    | 0.470184 |
| EX_malt_e   | 0.470184 |
| EX_lcts_e   | 0.411411 |


L-Ascorbate is available in the lab.

In [38]:
enhanced = ['EX_ascb__L_e']
enhanced_growth(model, CGXII_comp_14, enhanced)

Growth rate with additives: 0.7445
Doubling time with additives: 55.86 min


In [39]:
model = rg.io.load_model_cobra('../../models/iCstr1054FB23.xml')
enhanced_growth(model, CGXII_comp_14, CasA)

Growth rate with additives: 0.8624
Doubling time with additives: 48.23 min


In [43]:
CGXII_base = CGXII_comp_14.copy()
for metab, flux in CGXII_comp_14.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base['EX_cobalt2_e'] = 10.0

In [44]:
model = rg.io.load_model_cobra('../../models/iCstr1054FB23.xml')
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.6354
Doubling time with additives: 65.45 min


## Strain 1116

In [45]:
model = rg.io.load_model_cobra('../../models/iCstr1116FB23.xml')
CGXII_comp_17 = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_comp_17.csv').set_index('exchange').to_dict()['flux']

In [46]:
print(model.id)
enh = find_additives(model, CGXII_comp_17)
print(enh.head().to_markdown(index=False))

iCstr1116FB23
Growth rate without additives: 0.0
Doubling time without additives: inf min


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_13222/4155444981.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


| exchange    |     diff |
|:------------|---------:|
| EX_s_e      | 0.622009 |
| EX_cgly_e   | 0.613182 |
| EX_cys__L_e | 0.612059 |
| EX_h2s_e    | 0.611861 |
| EX_gthrd_e  | 0.610023 |


No simple additives found since there is not growth just on the CGXII medium even under the addition of `pnto_R`, `nmn` and `cobalt`. The next test is to try and find essential exchanges using refineGEMs and add those to the medium.

In [47]:
ess = rg.growth.get_essential_reactions_via_bounds(model)
for entry in ess:
    CGXII_comp_17.update({entry:10.0})

In [48]:
enh = find_additives(model, CGXII_comp_17)
print(enh.head().to_markdown(index=False))

Growth rate without additives: -0.0
Doubling time without additives: -1.8774713838470752e+30 min
| exchange    |     diff |
|:------------|---------:|
| EX_s_e      | 0.622009 |
| EX_cgly_e   | 0.613182 |
| EX_cys__L_e | 0.612059 |
| EX_h2s_e    | 0.611861 |
| EX_gthrd_e  | 0.610023 |


Still no reliable growth value. Next test is to add Casamino acids using the media definition given in refineGEMs.

In [49]:
model = rg.io.load_model_cobra('../../models/iCstr1116FB23.xml')
enhanced_growth(model, CGXII_comp_17, CasA)

Growth rate with additives: 0.7301
Doubling time with additives: 56.96 min


In [50]:
CGXII_comp_17 = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_comp_17.csv').set_index('exchange').to_dict()['flux']
for ex in CasA:
    CGXII_comp_17.update({ex:10.0})

In [51]:
enh = find_additives(model, CGXII_comp_17)
enh.to_csv('../../analysis/additives/Cstr_17_adds.csv', index=False)
print(enh.head().to_markdown(index=False))

Growth rate without additives: 0.7301
Doubling time without additives: 56.96 min
| exchange    |     diff |
|:------------|---------:|
| EX_malthp_e | 2.25677  |
| EX_raffin_e | 0.796507 |
| EX_tre_e    | 0.531005 |
| EX_malt_e   | 0.531005 |
| EX_sucr_e   | 0.531005 |


Strain 17 should be able to grow on CGXII under the addition of Casamino Acids.

In [52]:
model = rg.io.load_model_cobra('../../models/iCstr1116FB23.xml')
enhanced_growth(model, CGXII_comp_17, CasA)

Growth rate with additives: 0.7301
Doubling time with additives: 56.96 min


In [55]:
CGXII_base = CGXII_comp_17.copy()
for metab, flux in CGXII_comp_17.items():
    if metab not in CGXII.keys():
        CGXII_base.pop(metab)
CGXII_base['EX_cobalt2_e'] = 10.0
CGXII_base['EX_nmn_e'] = 10.0

In [56]:
model = rg.io.load_model_cobra('../../models/iCstr1116FB23.xml')
enhanced_growth(model, CGXII_base, CasA)

Growth rate with additives: 0.0
Doubling time with additives: inf min


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_13222/2110354515.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


## All models with CasA

In [6]:
CasA = rg.io.load_all_media_from_db('../../../refinegems/data/media_db.csv')[9]['BiGG_EX'].to_list()
modelpaths = ['../../models/iCstr1054FB23.xml', '../../models/iCstr1197FB23.xml', '../../models/iCstr1115FB23.xml', '../../models/iCstr1116FB23.xml']
cmods = [rg.io.load_model_cobra(path) for path in modelpaths]
cgx_paths = ['../../analysis/CGXII_base_medium/CGXII_comp_14.csv', '../../analysis/CGXII_base_medium/CGXII_comp_15.csv', '../../analysis/CGXII_base_medium/CGXII_comp_16.csv', '../../analysis/CGXII_base_medium/CGXII_comp_17.csv']
cgx_bases = [pd.read_csv(path).set_index('exchange').to_dict()['flux'] for path in cgx_paths]
cgx_co = pd.read_csv('../../analysis/CGXII_base_medium/CGXII_co.csv').set_index('exchange').to_dict()['flux']

In [7]:
for model, medium in zip(cmods, cgx_bases):
    print(model.id)
    enhanced_growth(model, cgx_co, CasA)

iCstr1054FB23
Growth rate with additives: 0.6354
Doubling time with additives: 65.45 min
iCstr1197FB23


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_12673/3608260857.py:52: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60
/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_12673/3608260857.py:52: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60


Growth rate with additives: 0.0
Doubling time with additives: inf min
iCstr1115FB23
Growth rate with additives: 0.0
Doubling time with additives: inf min
iCstr1116FB23
Growth rate with additives: 0.0
Doubling time with additives: inf min


/var/folders/s0/m197rw591kxcl13y56pqhf9m0000gs/T/ipykernel_12673/3608260857.py:52: RuntimeWarning: divide by zero encountered in double_scalars
  dt = (np.log(2) / sol.objective_value) * 60
